In [15]:
import pandas as pd
from amseg.amharicSegmenter import AmharicSegmenter
import sentencepiece as spm
import emoji
import re

In [16]:
# Load the telegram dataset
file_path = 'C:/Users/elbet/OneDrive/Desktop/Ten/week5/Telegram-based-e-commerce/telegram_data.csv'
df = pd.read_csv(file_path)

In [17]:
df

,Channel Title,Channel Username,ID,Message,Date,Media Path
0,Fashion tera,@Fashiontera,3966,NaN,2024-09-24 14:34:31+00:00,photos\@Fashiontera_3966.jpg
1,Fashion tera,@Fashiontera,3965,NaN,2024-09-24 14:34:31+00:00,photos\@Fashiontera_3965.jpg
2,Fashion tera,@Fashiontera,3964,〰️〰️〰️〰️〰️ \nCLASICO\n🔸🔸🔸🔸🔸🔸\nPrice 2200( No ...,2024-09-24 14:34:31+00:00,photos\@Fashiontera_3964.jpg
3,Fashion tera,@Fashiontera,3963,NaN,2024-09-21 18:56:46+00:00,photos\@Fashiontera_3963.jpg
4,Fashion tera,@Fashiontera,3962,〰️〰️〰️〰️〰️〰️〰️\nPuma \nMade in Vietnam \n🔸🔸🔸🔸🔸...,2024-09-18 15:26:06+00:00,photos\@Fashiontera_3962.jpg
...,...,...,...,...,...,...
2613,Fashion tera,@Fashiontera,96,Nikon \nD9 Digital Camera\nPrice 16000\nContac...,2018-07-05 20:11:35+00:00,photos\@Fashiontera_96.jpg
2614,Fashion tera,@Fashiontera,95,"Vans Leather\nMade in Vietnam\nSize 40, 41\nPr...",2018-07-05 19:50:38+00:00,photos\@Fashiontera_95.jpg
2615,Fashion tera,@Fashiontera,94,Samsung TV\nCurved Full HD \n55 Inch\nPrice 36...,2018-07-05 19:06:11+00:00,photos\@Fashiontera_94.jpg
2616,Fashion tera,@Fashiontera,92,Rebook\nMade in Vietnam\nSize 41\nPrice 1350\n...,2018-07-05 18:39:16+00:00,photos\@Fashiontera_92.jpg


In [18]:
print('Checking for NaN values in the message column')
nan_count = df['Message'].isnull().sum()
print(f'Number of NaN values in Message column: {nan_count}')

Checking for NaN values in the message column
Number of NaN values in Message column: 698


In [19]:
df = df.dropna(subset=['Message'])

#print the shape of the dataset after dropping NaN values in the "Message" columns
print(f"Dataset shape after dropping NaN values in 'Message' column: {df.shape}")

Dataset shape after dropping NaN values in 'Message' column: (1920, 6)


In [20]:
df.head()

,Channel Title,Channel Username,ID,Message,Date,Media Path
2,Fashion tera,@Fashiontera,3964,〰️〰️〰️〰️〰️ \nCLASICO\n🔸🔸🔸🔸🔸🔸\nPrice 2200( No ...,2024-09-24 14:34:31+00:00,photos\@Fashiontera_3964.jpg
4,Fashion tera,@Fashiontera,3962,〰️〰️〰️〰️〰️〰️〰️\nPuma \nMade in Vietnam \n🔸🔸🔸🔸🔸...,2024-09-18 15:26:06+00:00,photos\@Fashiontera_3962.jpg
8,Fashion tera,@Fashiontera,3950,New year Discount \n🌼🌼🌼🌼🌼🌼🌼🌼🌼\n〰️〰️〰️〰️〰️ \n🔸🔸...,2024-09-08 16:08:15+00:00,photos\@Fashiontera_3950.jpg
19,Fashion tera,@Fashiontera,3939,〰️〰️〰️〰️〰️ \nSkechers \nMade in Vietnam \n🔸🔸🔸🔸...,2024-09-08 15:45:52+00:00,photos\@Fashiontera_3939.jpg
31,Fashion tera,@Fashiontera,3915,〰️〰️〰️〰️〰️〰️〰️\nReebok \nMade in Vietnam \n🔸🔸🔸...,2024-09-01 19:38:25+00:00,photos\@Fashiontera_3915.jpg


In [21]:
message_df=df['Message']
message_df

2       〰️〰️〰️〰️〰️ \nCLASICO\n🔸🔸🔸🔸🔸🔸\nPrice  2200( No ...
4       〰️〰️〰️〰️〰️〰️〰️\nPuma \nMade in Vietnam \n🔸🔸🔸🔸🔸...
8       New year Discount \n🌼🌼🌼🌼🌼🌼🌼🌼🌼\n〰️〰️〰️〰️〰️ \n🔸🔸...
19      〰️〰️〰️〰️〰️ \nSkechers \nMade in Vietnam \n🔸🔸🔸🔸...
31      〰️〰️〰️〰️〰️〰️〰️\nReebok \nMade in Vietnam \n🔸🔸🔸...
                              ...                        
2612                                             Sold out
2613    Nikon \nD9 Digital Camera\nPrice 16000\nContac...
2614    Vans Leather\nMade in Vietnam\nSize 40, 41\nPr...
2615    Samsung TV\nCurved Full HD \n55 Inch\nPrice 36...
2616    Rebook\nMade in Vietnam\nSize 41\nPrice 1350\n...
Name: Message, Length: 1920, dtype: object

In [22]:
#function to remove emojis
def remove_emoji(text):
    return emoji.replace_emoji(text, replace='')

# Function to remove symbols and non-alphabetic characters
def remove_symbols(text):
    # Replace everything except letters (including unicode letters) and spaces
    return re.sub(r'[^A-Za-z0-9ሀ-ፐ\s]+', '', text)

#Apply the function to the 'message' column
message_df['cleaned_message'] = df['Message'].apply(remove_emoji).apply(remove_symbols)

C:\Users\elbet\AppData\Local\Temp\ipykernel_6656\4270856608.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  message_df['cleaned_message'] = df['Message'].apply(remove_emoji).apply(remove_symbols)
C:\Users\elbet\AppData\Local\Temp\ipykernel_6656\4270856608.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  message_df['cleaned_message'] = df['Message'].apply(remove_emoji).apply(remove_symbols)


In [23]:
df.to_csv('clean_data_csv')

In [24]:
# Extract the messages for further processing
tokens = message_df['cleaned_message'].tolist()

In [25]:
# Initialize the Amharic segmenter
sent_punct = []
word_punct = []
segmenter = AmharicSegmenter(sent_punct, word_punct)

# Function to check if the word contains Amharic characters
def contains_amharic(word):
    # Check if any character in the word is within the Amharic Unicode block (U+1200 to U+137F)
    return any(0x1200 <= ord(char) <= 0x137F for char in word)

# Function to segment tokens for Amharic
# Function to segment Amharic text (with type checking)
def segment_amharic_text(segmenter, tokens):
    segmented_tokens = []
    for word in tokens:
        if isinstance(word, str):  # Ensure the token is a string
            if contains_amharic(word):
                tokenized_word = segmenter.amharic_tokenizer(word)
            else:
                tokenized_word = word  # Leave non-Amharic words as is
            segmented_tokens.append(tokenized_word)
    return segmented_tokens

# Segment Amharic tokens
segmented_tokens = segment_amharic_text(segmenter, tokens)

# Display the first 20 segmented results
display("Segmented Amharic Tokens:", segmented_tokens[:20])

'Segmented Amharic Tokens:'

[['\nCLASICO\n\nPrice',
  '2200',
  'No',
  'gift',
  'box\nFree',
  'Delivery\nInbox',
  'Hiwe5266\nስልክ',
  '251945355266\n\n',
  'ፋሽን',
  'ተራ',
  'Fashion',
  'Tera',
  '\n',
  '\nአድራሻ',
  'አዲስ',
  'አበባ',
  'ጦር',
  'ሀይሎች',
  'ድሪም',
  'ታወር',
  '2ተኛ',
  'ፎቅ\nቢሮ',
  'ቁጥር',
  '205\n'],
 ['\nPuma',
  '\nMade',
  'in',
  'Vietnam',
  '\n\nSize',
  '404143\nPrice',
  '3600\nFree',
  'Delivery\nInbox',
  'Hiwe5266\nስልክ',
  '251945355266\n\n',
  'ፋሽን',
  'ተራ',
  'Fashion',
  'Tera',
  '\n',
  '\nአድራሻ',
  'አዲስ',
  'አበባ',
  'ጦር',
  'ሀይሎች',
  'ድሪም',
  'ታወር',
  '2ተኛ',
  'ፎቅ\nቢሮ',
  'ቁጥር',
  '205\n'],
 ['New',
  'year',
  'Discount',
  '\n\n',
  '\n\nInbox',
  'Hiwe5266\nስልክ',
  '251945355266\n\n',
  'ፋሽን',
  'ተራ',
  'Fashion',
  'Tera',
  '\n',
  '\nአድራሻ',
  'አዲስ',
  'አበባ',
  'ጦር',
  'ሀይሎች',
  'ድሪም',
  'ታወር',
  '2ተኛ',
  'ፎቅ\nቢሮ',
  'ቁጥር',
  '205\n\n'],
 ['\nSkechers',
  '\nMade',
  'in',
  'Vietnam',
  '\n\nSize',
  '4243\nPrice',
  '3100\nFree',
  'Delivery\nInbox',
  'Hiwe5266\nስልክ',
  '2519453

In [26]:
# Save the final tokens to a CSV file
output_df = pd.DataFrame({'Token': segmented_tokens})
output_df.to_csv('C:/Users/elbet/OneDrive/Desktop/Ten/week5/Telegram-based-e-commerce/final_telegram_tokens.csv', index=False)

In [27]:
# Display tokens from index 50 to 80
print("Display Results from Index 50 to 80:")
for token in segmented_tokens[50:80]:
    display(f"{token}")

Display Results from Index 50 to 80:


"['\\nSpeed', 'Couple', 'Watch', '\\n\\nPrice', '3800\\nFree', 'Delivery\\nInbox', 'Hiwe5266\\nስልክ', '251945355266\\n\\n', 'ፋሽን', 'ተራ', 'Fashion', 'Tera', '\\n', '\\nአድራሻ', 'አዲስ', 'አበባ', 'ጦር', 'ሀይሎች', 'ድሪም', 'ታወር', '2ተኛ', 'ፎቅ\\nቢሮ', 'ቁጥር', '205\\n']"

"['\\nNike', 'SB\\nMade', 'in', 'Vietnam', '\\n\\nSize', '404243\\nPrice', '2950\\nFree', 'Delivery\\nInbox', 'Hiwe5266\\nስልክ', '251945355266\\n\\n', 'ፋሽን', 'ተራ', 'Fashion', 'Tera', '\\n', '\\nአድራሻ', 'አዲስ', 'አበባ', 'ጦር', 'ሀይሎች', 'ድሪም', 'ታወር', '2ተኛ', 'ፎቅ\\nቢሮ', 'ቁጥር', '205\\n']"

"['\\nCHANEL', 'ROLEX', '\\n\\nPrice', '2000', 'No', 'gift', 'box\\nFree', 'Delivery\\nInbox', 'Hiwe5266\\nስልክ', '251945355266\\n\\n', 'ፋሽን', 'ተራ', 'Fashion', 'Tera', '\\n', '\\nአድራሻ', 'አዲስ', 'አበባ', 'ጦር', 'ሀይሎች', 'ድሪም', 'ታወር', '2ተኛ', 'ፎቅ\\nቢሮ', 'ቁጥር', '205\\n']"

"['\\nSkechers', '\\nMade', 'in', 'Vietnam', '\\n\\nSize', '40414243\\nPrice', '2800\\nFree', 'Delivery\\nInbox', 'Hiwe5266\\nስልክ', '251945355266\\n\\n', 'ፋሽን', 'ተራ', 'Fashion', 'Tera', '\\n', '\\nአድራሻ', 'አዲስ', 'አበባ', 'ጦር', 'ሀይሎች', 'ድሪም', 'ታወር', '2ተኛ', 'ፎቅ\\nቢሮ', 'ቁጥር', '205\\n']"

...

"['\\n', 'Air', 'Jordan', '1\\xa0', 'Low', 'Travis', 'scott\\nMade', 'in', 'Vietnam', '\\nSize', '404243\\nPrice', '2600\\nFree', 'Delivery\\nInbox', 'Hiwe5266\\nስልክ', '251945355266\\n\\n', 'ፋሽን', 'ተራ', 'Fashion', 'Tera\\n', '\\nአድራሻ', 'አዲስ', 'አበባ', 'ጦር', 'ሀይሎች', 'ድሪም', 'ታወር', '2ተኛ', 'ፎቅ\\nቢሮ', 'ቁጥር', '205\\n']"

"['\\nBRAND', 'Watches\\nFree', 'Delivery\\nInbox', 'Hiwe5266\\nስልክ', '251945355266\\n\\n', 'ፋሽን', 'ተራ', 'Fashion', 'Tera\\n', '\\nአድራሻ', 'አዲስ', 'አበባ', 'ጦር', 'ሀይሎች', 'ድሪም', 'ታወር', '2ተኛ', 'ፎቅ\\nቢሮ', 'ቁጥር', '205\\n\\n']"

"['\\n', 'Air', 'Jordan', '4', 'LV\\nMade', 'in', 'Vietnam', '\\nPrice', '32003500\\nFree', 'Delivery\\nInbox', 'Hiwe5266\\nስልክ', '251945355266\\n\\n', 'ፋሽን', 'ተራ', 'Fashion', 'Tera\\n', '\\nአድራሻ', 'አዲስ', 'አበባ', 'ጦር', 'ሀይሎች', 'ድሪም', 'ታወር', '2ተኛ', 'ፎቅ\\nቢሮ', 'ቁጥር', '205\\n']"

"['Air', 'Jordan', '\\nMade', 'in', 'Vietnam', '\\nSize', '40424344\\nPrice', '2300\\nFree', 'Delivery\\nInbox', 'Hiwe5266\\nስልክ', '251945355266\\n\\n', 'ፋሽን', 'ተራ', 'Fashion', 'Tera\\n', '\\nአድራሻ', 'አዲስ', 'አበባ', 'ጦር', 'ሀይሎች', 'ድሪም', 'ታወር', '2ተኛ', 'ፎቅ\\nቢሮ', 'ቁጥር', '205\\n']"

"['Air', 'Jordan', '\\nMade', 'in', 'Vietnam', '\\nSize', '40424344\\nPrice', '2300\\nFree', 'Delivery\\nInbox', 'Hiwe5266\\nስልክ', '251945355266\\n\\n', 'ፋሽን', 'ተራ', 'Fashion', 'Tera\\n', '\\nአድራሻ', 'አዲስ', 'አበባ', 'ጦር', 'ሀይሎች', 'ድሪም', 'ታወር', '2ተኛ', 'ፎቅ\\nቢሮ', 'ቁጥር', '205\\n']"

In [28]:
# Convert all tokens to strings before writing to the file
tokens = [str(token) for token in tokens]

# Write the tokens to a temporary text file for SentencePiece training
with open('temp_text.txt', 'w', encoding='utf-8') as f:
    f.write("\n".join(tokens))
